In [ ]:
!pip install ydata-profiling
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install xgboost
!pip install prophet
!pip install catboost


In [ ]:
# is Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import RMSprop, Adam
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras import backend as K
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import shutil
from datetime import date, timedelta
from datetime import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from prophet import Prophet
import pickle
from keras.models import load_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import TimeSeriesSplit
import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

# Загрузка словаря из файла
with open('historical_lags_season.pkl', 'rb') as f:
    season_dfs = pickle.load(f)


In [ ]:
season_2 = season_dfs['lag_data_season_2']

In [ ]:

# Загрузка словаря из файла
with open('/kaggle/input/models-mapping/2_mapping.pkl', 'rb') as f:
    mapping_2 = pickle.load(f)


In [ ]:
st_id_encoder = {v: k for k, v in mapping_2['st_id_2_encoded'].items()}
pr_sku_id_encoder = {v: k for k, v in mapping_2['pr_sku_id_2_encoded'].items()}


In [ ]:
def wape_loss(y_true, y_pred):
    y_true_float = K.cast(y_true, dtype='float32')  # Преобразовать y_true в float32
    error = K.abs(y_true_float - y_pred)
    denominator = K.abs(y_true_float) + K.abs(y_pred)
    return K.sum(error) / K.sum(denominator)

In [ ]:
custom_objects = {'wape_loss': wape_loss}


In [ ]:
model_2 = tf.keras.models.load_model('/kaggle/input/models-mapping/2.h5', custom_objects=custom_objects)


In [ ]:
test_end_date = season_2['date'].max()
    # Вычисляем начальную дату, отсчитывая от конца
test_start_date = test_end_date - timedelta(days=days_to_keep_in_test - 1)
    # Фильтруем данные для тестовой выборки
data_new_test = season_2[(season_2['date'] >= test_start_date) & (season_2['date'] <= test_end_date)]
    # Фильтруем данные для тренировочной выборки (оставляем все, кроме тестовой части)


In [ ]:

data_new_test.drop(['date'],axis=1,inplace=True)
features_lag_test = data_new_test.drop('pr_sales_in_units',axis=1)
target_lag_test = data_new_test.pr_sales_in_units

In [ ]:
features_lag_test['st_id_encoded'] = features_lag_test['st_id'].map(st_id_encoder)
features_lag_test['pr_sku_id_encoded'] = features_lag_test['pr_sku_id'].map(pr_sku_id_encoder)

In [ ]:
features_lag_test['st_id_encoded'].fillna(value=999,inplace=True)
features_lag_test['pr_sku_id_encoded'].fillna(value=9999,inplace=True)

In [ ]:
features_lag_test = features_lag_test.drop(['st_id','pr_sku_id'],axis=1)

In [ ]:
features_test_tensor = tf.convert_to_tensor(features_lag_test.values, dtype=tf.float32)
target_test_tensor = tf.convert_to_tensor(target_lag_test.values, dtype=tf.int64)

In [ ]:
test_loss = model_2.evaluate(np.expand_dims(features_test_tensor, axis=-1), target_test_tensor)

In [ ]:
predictions = model_2.predict(features_test_tensor)
